In [1]:
from os import cpu_count
from qiskit_aer import AerSimulator
from qiskit import*
from tqdm import tqdm
import pylatexenc
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter,MultipleLocator,LinearLocator
from matplotlib import cm
import numpy as np
from numpy import pi,sin,cos
from numpy.random import randint,choice
from qiskit.quantum_info import Statevector
from multiprocessing import Pool
from datetime import datetime

In [2]:
def parallel(vector:list,func):
    with Pool(int(cpu_count()/2)) as p:
        return p.map(func,vector)

## Quantum Circuits Functions

In [3]:
def circuit_singlet_creator(angle_between_analyzers:float):
    global_shift = pi # alpha angle between analizer 0 and 1 for alice, angle between x and analizer 0 for bob
    # ---------------------------------------
    q = QuantumCircuit(2,2)
    q.x(0); q.x(1); q.h(0); q.rz(relative_phase,0); q.cx(0,1) # singlet state with phase
    q.barrier()
    basis_alice = choice([0,1,2]); basis_bob = choice([1,2,3])
    basis_list = {0:0,1:global_shift,2:(global_shift+angle_between_analyzers),3:3*pi/4}
    q.ry(basis_list[basis_alice],0); q.ry(basis_list[basis_bob],1)
    q.barrier()
    q.measure(0,0); q.measure(1,1)
    basis = str(basis_alice)+str(basis_bob)
    return q,basis
    

In [4]:
def circuit_list_vector_creator(n_states:int,angle_between_analyzers:float):
    circs_basis_vector = parallel([angle_between_analyzers for k in range(n_states)],circuit_singlet_creator)
    return circs_basis_vector

In [5]:
def exec(q:QuantumCircuit):
    simulator = AerSimulator()
    circ = transpile(q, simulator)
    result = simulator.run(circ,shots=1).result().get_counts()
    return list(dict(result).keys())[0]

## Post Execution Functions

In [6]:
def info_selector(circs_basis_vector:list):
    N = len(circs_basis_vector)
    circuits = list()
    basis=list()
    for k in range(N):
        circuits.append(circs_basis_vector[k][0])
        basis.append(circs_basis_vector[k][1])
    return [circuits,basis]

def probability_computation(basis_vector:list,result_vector:list):
    N_same_basis = 0
    N_anticorrelation = 0
    N_states = len(basis_vector)
    for k in range(N_states):
        if basis_vector[k] == '11' or basis_vector[k] == '22':
            N_same_basis += 1
            if result_vector[k] == '10' or result_vector[k] == '01':
                N_anticorrelation += 1
            else:
                pass
        else:
            pass
    return N_anticorrelation/N_same_basis

## Run Function

In [7]:
def local_run (n_states:int,angle_1_2:float):
    info = circuit_list_vector_creator(n_states=n_states,angle_between_analyzers=angle_1_2)
    circuit_list, basis_list = info_selector(info)
    del info
    result = parallel(vector=circuit_list,func=exec)
    result = [result[k]for k in range(len(result))]
    del circuit_list
    fidelity = probability_computation(basis_vector=basis_list,result_vector=result)    
    return fidelity

## plots

In [8]:
def plots (x:list,y_set:list,n_states:int,n_execs:int,relative_phase_set:list):
    f,ax=plt.subplots(1)
    n_y = len(y_set)
    for k in range(n_y):
        phase = round(relative_phase_set[k]/pi,1)
        ax.plot(x/pi,y_set[k],label=f"$\\phi$ = {phase}$\\pi$")
        ax.legend(bbox_to_anchor=(1.3, 1),fancybox=True,title="relative phase $\\phi$")
    ax.set_ylim([0,1.5])
    ax.set_xlim([min(x/pi),max(x/pi)])
    ax.xaxis.set_major_formatter(FormatStrFormatter('%g $\\pi$'))
    ax.xaxis.set_major_locator(MultipleLocator(base=1.0))
    ax.set_xlabel("angle between coincident analyzers",fontsize=12)
    ax.set_ylabel("N_anticorrelations/N_same_basis",fontsize=12)
    plt.title(f"number of states transmitted = {n_states:1.0e}\nnumber of E91 runs = {n_execs}",fontsize=12)
    
    plt.show()

# main 

In [9]:
def main(n_states:int,n_execs:int):
    fidelity = list()
    for k in range(n_execs):
        fidelity.append(local_run(n_states=n_states,angle_1_2=angle_between_analyzers[k]))
    return np.array(fidelity)
    
def save_info_to_csv(column:list,backend_name,time_str):
    file_name = f"data_{backend_name}_{time_str}.csv"
    string = f"{column[0]}"
    for data in column[1]:
        string += f",{data}"
    with open(file_name, "a") as file:
        file.write("\n" + string)

In [ ]:
states = int(1.5e4)
execs = int(1e2)
n_phase = int(1e2)
phase = np.linspace(0,2*pi,n_phase)

global angle_between_analyzers
angle_between_analyzers = np.linspace(0,2*pi,execs)

time_now = datetime.now().strftime("%d_%m_%Y_%H_%M")
for rel_phase in tqdm(phase,desc="progress: "):
    global relative_phase
    relative_phase=rel_phase # phase between bell pair state
    #---------------------------------------------------
    out = main(n_states=states,n_execs=execs)
    if rel_phase != 0.:
        save_info_to_csv(column=[rel_phase,out],backend_name="aer_simulator",time_str=time_now)
    else:
        save_info_to_csv(column=['#',angle_between_analyzers],backend_name="aer_simulator",time_str=time_now)
        save_info_to_csv(column=[rel_phase,out],backend_name="aer_simulator",time_str=time_now)
    #---------------------------------------------------

progress:   0%|                                         | 0/100 [00:00<?, ?it/s]

# Plots

In [ ]:
plots(x=angle,y_set=result,n_states=states,n_execs=execs,relative_phase_set=phase)


In [ ]:
fg = plt.figure(figsize=(8,8))
plt.imshow(result, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.contour(result)

plt.show()

In [ ]:
X,Y = np.meshgrid(angle/pi,phase/pi)
f,axs = plt.subplots(subplot_kw={'projection':'3d'},figsize=(8,8))
surf = axs.plot_surface(X,Y,np.array(result),cmap=cm.magma)
axs.set_xlabel("Angle between polarizers",fontsize=12)
axs.set_ylabel("Relative Phase of Bell State",fontsize=12)
axs.set_zlabel("N_anticorrelations/N_same_basis",fontsize=12)
axs.set_zlim([0,1.3])
axs.xaxis.set_major_formatter(FormatStrFormatter('%g $\\pi$'))
axs.xaxis.set_major_locator(MultipleLocator(base=1.0))
axs.yaxis.set_major_formatter(FormatStrFormatter('%g $\\pi$'))
axs.yaxis.set_major_locator(MultipleLocator(base=1.0))
f.colorbar(surf, shrink=0.5, aspect=5)
plt.show()